# Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
np.random.seed(1337)
from tensorflow import keras 
from keras import Sequential
from keras.utils import Sequence
from tensorflow.keras.utils import Sequence
from keras.layers import LSTM, Dense, Masking
import numpy as np
import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, Input, concatenate, Layer, Lambda, Dropout, Activation
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
import tensorflow as tf
import tensorflow_hub as hub

Using TensorFlow backend.


# Loading The Data

In [2]:
train_raw = pd.read_csv('train_6128.csv')
train_raw.head()

,Unnamed: 0,Text,MLclass
0,5860,1\r\ni SUPPLIES & SOLUTIONS FOR INDUSTRY\r\nww...,single
1,6007,"\r\nC. BEAN TRANSPORT, INC.\r\n(VAN DIVISION)\...",single
2,1192,"support@almo.com Tuesday, June 10, 2014 1:46 P...",single
3,6214,"1\r\nMuller Martini 456 Wheeler Rd Hauppauge, ...",single
4,4231,"INVOICE\r\nAndrews\r\nAndrews Paperboard, Inc....",single


In [3]:
train_raw.shape

(6128, 3)

# Preprocessing Data

Select non null:

In [4]:
train_raw = train_raw[train_raw.Text.notnull()]
train_raw.shape

(6127, 3)

In [5]:
train_raw.Text.apply(lambda x: len(x.split())).plot(kind='hist')

<AxesSubplot:ylabel='Frequency'>

In [6]:
train_raw['len_txt'] =train_raw.Text.apply(lambda x: len(x.split()))
train_raw.describe()

,Unnamed: 0,len_txt
count,6127.000000,6127.000000
mean,3843.316631,200.015668
std,2201.616986,118.279703
min,1.000000,1.000000
25%,1957.500000,125.000000
50%,3845.000000,181.000000
75%,5744.000000,252.000000
max,7659.000000,2317.000000


In [7]:
train_raw.shape

(6127, 4)

Select only the row with number of words greater than 250:

In [8]:
train_raw = train_raw[train_raw.len_txt >249]
train_raw.shape

(1582, 4)

Group similar products

In [9]:
for l in np.unique(train_raw['MLclass']):
  print(l)

first
intermediate
last
single


In [10]:
train_raw['MLclass'].value_counts().sort_values(ascending=False).plot(kind='bar')

<AxesSubplot:ylabel='Frequency'>

In [11]:
train_raw=train_raw.rename(columns = {'Text':'text', 'MLclass':'label'})
train_raw.head()

,Unnamed: 0,text,label,len_txt
5,1513,GRAINGER\r\n11\r\nPAGE 1 OF 1\r\n7200 NW 37TH ...,single,286
8,7645,"ro to u >\r\n........ ?v. f {f. i. 0\t,\teea-i...",first,261
14,4012,Gordon\r\nFOOD SERVICE\r\nGordon Food Service ...,single,408
17,4094,PAGE 1\r\nGRAINGER\r\n11\r\n2775 S. 900 W.\r\n...,single,278
19,5230,INVOICE\r\nREMIT TO: 37302 Eagle Wav - Chicago...,single,290


In [12]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
train_raw['label'] = LE.fit_transform(train_raw['label'])
train_raw.head()

,Unnamed: 0,text,label,len_txt
5,1513,GRAINGER\r\n11\r\nPAGE 1 OF 1\r\n7200 NW 37TH ...,3,286
8,7645,"ro to u >\r\n........ ?v. f {f. i. 0\t,\teea-i...",0,261
14,4012,Gordon\r\nFOOD SERVICE\r\nGordon Food Service ...,3,408
17,4094,PAGE 1\r\nGRAINGER\r\n11\r\n2775 S. 900 W.\r\n...,3,278
19,5230,INVOICE\r\nREMIT TO: 37302 Eagle Wav - Chicago...,3,290


In [13]:
len(np.unique(train_raw['label']))

4

In [14]:
train_raw = train_raw[["text", "label"]]

In [15]:
train_raw.head()

,text,label
5,GRAINGER\r\n11\r\nPAGE 1 OF 1\r\n7200 NW 37TH ...,3
8,"ro to u >\r\n........ ?v. f {f. i. 0\t,\teea-i...",0
14,Gordon\r\nFOOD SERVICE\r\nGordon Food Service ...,3
17,PAGE 1\r\nGRAINGER\r\n11\r\n2775 S. 900 W.\r\n...,3
19,INVOICE\r\nREMIT TO: 37302 Eagle Wav - Chicago...,3


In [16]:
train = train_raw.copy()

In [17]:
train = train.reindex(np.random.permutation(train.index))
train.head()

,text,label
3377,ORIGINAL INVOICE\r\nD.U.N.S\r\n>\r\n>\r\nc_\r\...,3
5218,{PRAXAIR\r\nPRAXAIR DISTRIBUTION INC 3607 SOUT...,3
4045,] REMIT PAYMENT TO:\r\nCDW Direct\r\nPO Box 75...,3
3222,"lYtlLBf\r\nYELLOW TRANSPORTATION, INC. (YFSY) ...",3
5474,Account Statement\r\nAudi\r\nFinancial Service...,3


Clean the text columns

In [18]:
import re
def clean_txt(text):
  text = re.sub("'", "",text)
  text=re.sub("(\\W)+"," ",text)    
  return text

In [19]:
train['text']  = train.text.apply(clean_txt)
train.head()

,text,label
3377,ORIGINAL INVOICE D U N S c_ MOHAWK CARPET MOHA...,3
5218,PRAXAIR PRAXAIR DISTRIBUTION INC 3607 SOUTH M...,3
4045,REMIT PAYMENT TO CDW Direct PO Box 75723 Chic...,3
3222,lYtlLBf YELLOW TRANSPORTATION INC YFSY P 0 BOX...,3
5474,Account Statement Audi Financial Services Summ...,3


In [20]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=35)
train.head()

,text,label
3026,CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRAN...,3
5691,Wichita Kansas Topeka Kansas Overland Park Kan...,0
4386,NETSUITE OMSTSUM NDUMirS 2955 Campus Drive Sui...,0
107,Invoice 943794 THE PRODUCE CONNECTION INC 2200...,3
1023,1 YELLOW TRANSPORTATION INC YFSY YELLOW P 0 BO...,3


In [21]:
train.reset_index(drop=True, inplace=True)
train.head(2)

,text,label
0,CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRAN...,3
1,Wichita Kansas Topeka Kansas Overland Park Kan...,0


In [22]:
val.reset_index(drop=True, inplace=True)
val.head(2)

,text,label
0,TEL 707 545 4540 FAX 707 545 5902 800 998 9070...,2
1,Invoice McMASTER CARR 630 600 3600 630 834 94...,0


In [23]:
val.shape, train.shape

((317, 2), (1265, 2))

In [24]:
#Installing BERT module
!pip install bert-tensorflow

In [25]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# Setting The Output Directory for BERT

In [26]:
# Set the output directory for saving model file
OUTPUT_DIR = './bert_news_category'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
#tf.io.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: ./bert_news_category *****


In [27]:
print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)
# print("Test Set Shape :", test.shape)

Training Set Shape : (1265, 2)
Validation Set Shape : (317, 2)


In [28]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# The list containing all the classes (train['SECTION'].unique())
label_list = [x for x in np.unique(train.label)]
label_list

[0, 1, 2, 3]

# Splitting the Data into smaller chunks

In [29]:
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:200]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_parcial))
  return l_total

In [30]:
train['text_split'] = train[DATA_COLUMN].apply(get_split)
train.head()

,text,label,text_split
0,CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRAN...,3,[CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRA...
1,Wichita Kansas Topeka Kansas Overland Park Kan...,0,[Wichita Kansas Topeka Kansas Overland Park Ka...
2,NETSUITE OMSTSUM NDUMirS 2955 Campus Drive Sui...,0,[NETSUITE OMSTSUM NDUMirS 2955 Campus Drive Su...
3,Invoice 943794 THE PRODUCE CONNECTION INC 2200...,3,[Invoice 943794 THE PRODUCE CONNECTION INC 220...
4,1 YELLOW TRANSPORTATION INC YFSY YELLOW P 0 BO...,3,[1 YELLOW TRANSPORTATION INC YFSY YELLOW P 0 B...


In [31]:
val['text_split'] = val[DATA_COLUMN].apply(get_split)
val.head(2)

,text,label,text_split
0,TEL 707 545 4540 FAX 707 545 5902 800 998 9070...,2,[TEL 707 545 4540 FAX 707 545 5902 800 998 907...
1,Invoice McMASTER CARR 630 600 3600 630 834 94...,0,[Invoice McMASTER CARR 630 600 3600 630 834 94...


In [32]:
train_l = []
label_l = []
index_l =[]
for idx,row in train.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['label'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(2578, 2578, 2578)

In [33]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in val.iterrows():
  for l in row['text_split']:
    val_l.append(l)
    val_label_l.append(row['label'])
    val_index_l.append(idx)
len(val_l), len(val_label_l), len(val_index_l)

(634, 634, 634)

The final dataset for training:

In [34]:
train_df = pd.DataFrame({DATA_COLUMN:train_l, LABEL_COLUMN:label_l})
train_df.head()

,text,label
0,CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRAN...,3
1,NQI SEE NOTES ITEMS 154634 AND 154636 BOOTHS O...,3
2,Wichita Kansas Topeka Kansas Overland Park Kan...,0
3,19 Review settlement correspondence and attach...,0
4,NETSUITE OMSTSUM NDUMirS 2955 Campus Drive Sui...,0


In [35]:
val_df = pd.DataFrame({DATA_COLUMN:val_l, LABEL_COLUMN:val_label_l})
val_df.head()

,text,label
0,TEL 707 545 4540 FAX 707 545 5902 800 998 9070...,2
1,3 43 31 1 1 I cs 18 ROLL COMPACT TISSUE CORELE...,2
2,Invoice McMASTER CARR 630 600 3600 630 834 942...,0
3,2 70 Each 13 50 3 4429K163 Low Pressure Brass ...,0
4,Glasgow Audi Unit 1 Sentinel Court Hi Kington ...,3


# BERT: Data Preprocessing 

Process the data for BERT

In [36]:
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [37]:
train_InputExamples

0       <bert.run_classifier.InputExample object at 0x...
1       <bert.run_classifier.InputExample object at 0x...
2       <bert.run_classifier.InputExample object at 0x...
3       <bert.run_classifier.InputExample object at 0x...
4       <bert.run_classifier.InputExample object at 0x...
                              ...                        
2573    <bert.run_classifier.InputExample object at 0x...
2574    <bert.run_classifier.InputExample object at 0x...
2575    <bert.run_classifier.InputExample object at 0x...
2576    <bert.run_classifier.InputExample object at 0x...
2577    <bert.run_classifier.InputExample object at 0x...
Length: 2578, dtype: object

In [38]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRANSPORTATION SERVICES INC DUNS 10 155 1257 FED TAX ID 94 2904084 INVOICE 781 456922 CUSTOMER SERVICE PHONE 1 800 755 2728 FAX 817 514 3700 CORRESPONDENCE OVERNIGHT MAIL PO BOX 982020 5555 RUFE SNOW DR SUITE 5515 N RICHLAND HILLS TX 76182 N RICHLAND HILLS TX 76180 VISIT US AT www con wav com FOR A COPY OF THE BILL OF LADING OR PROOF OF DELIVERY H30 000202 030000 COOPER LIGHTING ATTN TRAFFIC DEPT 1121 HIGHWAY 74 S PEACHTREE CITS GA 30269 3019 B I L l T O SHIPPER CONSIGNEE CUSTOMER NUMBERS SHIPMENT DATE HARTWIG EXHIBITIONS COOPER LIGHTING BOOTH 649 SNS5131 FOR COOPER BOOTH 649 XFREEMAN DECOR 16325 W RYERSON RD 1616 WASHINGTON ST 03 25 05 NEW BERLIN WI 53151 3629 BRAINTREE MA 02184 PCS DESCRIPTION OF ARTICLES AND MARKS WEIGHT lbs RATE CHARGES 4 PCS E XHTBITIGN MATER IALS NQI SEE NOTES ITEMS 154634 AND 154636 BOOTHS OR STALL S IN BOXES CRATES OR TRUNKS SEE NOTE 1546

# BERT: Loading the pre-trained model

In [39]:

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [ ]:
len(tokenizer.vocab.keys())

30522

In [ ]:
#Here is what the tokenised sample of the first training set observation looks like
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

UnparsedFlagAccessError: Trying to access flag --preserve_unused_tokens before flags were parsed.

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
MAX_SEQ_LENGTH = 200

In [ ]:
# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 2578


INFO:tensorflow:Writing example 0 of 2578


UnparsedFlagAccessError: Trying to access flag --preserve_unused_tokens before flags were parsed.

In [ ]:
! pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 67 kB 2.0 MB/s 
  Attempting uninstall: bert-tensorflow
    Found existing installation: bert-tensorflow 1.0.4
    Uninstalling bert-tensorflow-1.0.4:
      Successfully uninstalled bert-tensorflow-1.0.4


In [ ]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  CCM COniUftV CEHTRRL EXPRESS cwce CON WAY TRANSPORTATION SERVICES INC DUNS 10 155 1257 FED TAX ID 94 2904084 INVOICE 781 456922 CUSTOMER SERVICE PHONE 1 800 755 2728 FAX 817 514 3700 CORRESPONDENCE OVERNIGHT MAIL PO BOX 982020 5555 RUFE SNOW DR SUITE 5515 N RICHLAND HILLS TX 76182 N RICHLAND HILLS TX 76180 VISIT US AT www con wav com FOR A COPY OF THE BILL OF LADING OR PROOF OF DELIVERY H30 000202 030000 COOPER LIGHTING ATTN TRAFFIC DEPT 1121 HIGHWAY 74 S PEACHTREE CITS GA 30269 3019 B I L l T O SHIPPER CONSIGNEE CUSTOMER NUMBERS SHIPMENT DATE HARTWIG EXHIBITIONS COOPER LIGHTING BOOTH 649 SNS5131 FOR COOPER BOOTH 649 XFREEMAN DECOR 16325 W RYERSON RD 1616 WASHINGTON ST 03 25 05 NEW BERLIN WI 53151 3629 BRAINTREE MA 02184 PCS DESCRIPTION OF ARTICLES AND MARKS WEIGHT lbs RATE CHARGES 4 PCS E XHTBITIGN MATER IALS NQI SEE NOTES ITEMS 154634 AND 154636 BOOTHS OR STALL S IN BOXES CRATES OR TRUNKS SEE NOTE 154640 LOOSE WHEN SHIPMENT OVER 16 000 LB OR MORE 154630 CLASS 125 CON WAY 

UnparsedFlagAccessError: Trying to access flag --preserve_unused_tokens before flags were parsed.

# BERT: Creating A Multi-Class Classifier Model

In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  # with tf.Session() as sess:
  output_layer1 = bert_outputs["pooled_output"]
  # output_layer1 = 999
  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.8)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs, output_layer1)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg,
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs, output_layer) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels,
          'pooled_output': output_layer
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

NameError: name 'train_features' is not defined

In [ ]:
num_train_steps, len(label_list)

(1979, 10)

In [ ]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': '/bert_news_category', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1fc8615630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/bert_news_category', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1fc8615630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

# BERT: Fine Tuning Training & Evaluating

In [ ]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 1979 into /bert_news_category/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1979 into /bert_news_category/model.ckpt.


INFO:tensorflow:Loss for final step: 0.2288338.


INFO:tensorflow:Loss for final step: 0.2288338.


Training took time  0:15:36.286117


The accuracy for the fine tuned BERT model

In [ ]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


{'eval_accuracy': 0.8582935,
 'false_negatives': 116.0,
 'false_positives': 176.0,
 'global_step': 1979,
 'loss': 0.4725039,
 'true_negatives': 693.0,
 'true_positives': 6961.0}

In [ ]:
! pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.8.0-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.0 MB/s eta 0:00:00
  Using cached html5lib-0.9999999.tar.gz (889 kB)
  Preparing metadata (setup.py) ... done
  Using cached bleach-1.5.0-py2.py3-none-any.whl (17 kB)
  Created wheel for html5lib: filename=html5lib-0.9999999-py3-none-any.whl size=107234 sha256=1d594446e4626996adf9d409ced4a83dd759b3f297a233b9d50bb6e3d4c05898
  Stored in directory: /home/komleva/.cache/pip/wheels/dc/75/d2/3812ab8ba1fff9d3a0ed5bfc472174c9312055b1b80a94834b
Successfully built html5lib
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.1
    Uninstalling html5lib-1.1:
      Successfully uninstalled html5lib-1.1
  Attempting uninstall: bleach
    Found existing installation: bleach 4.0.0
    Uninstalling bleach-4.0.0:
      Successfully uninstalled bleach-4.0.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8

# BERT: Get The Vector Transformations from the Fine Tuned BERT

In [ ]:
# A method to get predictions
def getPrediction(in_sentences, type_output = "features"):
  #A list to map the actual labels to the predictions
  labels = np.unique(train['label'])
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  if type_output == "features":
    return [prediction['pooled_output'] for _,prediction in enumerate(predictions) ]
  else:
    return ([(sentence, prediction['probabilities'],
              prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)])


In [ ]:
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)
MAX_SEQ_LENGTH

200

In [ ]:
train_df.shape, val_df.shape

((31679, 2), (7946, 2))

Now extracting the representations:

In [ ]:
%%time
tr_emb = np.apply_along_axis(getPrediction, 0,np.array(train_df[DATA_COLUMN]))


CPU times: user 2min 35s, sys: 10.9 s, total: 2min 46s
Wall time: 6min 10s


In [ ]:
%%time
val_emb = np.apply_along_axis(getPrediction, 0,np.array(val_df[DATA_COLUMN]))
val_emb.shape

CPU times: user 43.4 s, sys: 2.76 s, total: 46.2 s
Wall time: 1min 37s


In [ ]:
val_emb.shape, tr_emb.shape

((7946, 768), (31679, 768))

and make the dataset for train and val:

In [ ]:
aux = -1
len_l = 0
train_x = {}
for l, emb in zip(index_l, tr_emb):
  if l in train_x.keys():
    train_x[l]  =np.vstack([train_x[l], emb])
  else:
    train_x[l] = [emb]

len(train_x.keys())


13713

In [ ]:
train_l_final = []
label_l_final = []
for k in train_x.keys():
  train_l_final.append(train_x[k])
  label_l_final.append(train.loc[k]['label'])

df_train = pd.DataFrame({'emb': train_l_final, 'label': label_l_final, })
df_train.head()

,emb,label
0,"[[0.848892, 0.05472551, -0.85581946, 0.4909455...",3
1,"[[0.4019187, 0.30476514, 0.034283057, -0.10456...",4
2,"[[0.9221226, -0.48432896, -0.9658792, 0.905419...",0
3,"[[-0.34883273, 0.5719657, -0.65025216, 0.69365...",3
4,"[[0.8565562, -0.18846692, -0.98491526, 0.75137...",2


In [ ]:
aux = -1
len_l = 0
val_x = {}

for l, emb in zip(val_index_l, val_emb):
  if l in val_x.keys():
    val_x[l]  =np.vstack([val_x[l], emb])
  else:
    val_x[l] = [emb]


val_l_final = []
vlabel_l_final = []
for k in val_x.keys():
  val_l_final.append(val_x[k])
  vlabel_l_final.append(val.loc[k]['label'])

df_val = pd.DataFrame({'emb': val_l_final, 'label': vlabel_l_final})
df_val.head()

,emb,label
0,"[[0.7906871, -0.3191274, -0.82637614, 0.760878...",0
1,"[[-0.18997923, 0.5434573, -0.8482759, 0.609339...",3
2,"[[-0.2863525, 0.47603318, -0.84779084, 0.70149...",3
3,"[[-0.55615234, 0.6234803, -0.7726333, -0.04120...",3
4,"[[0.24671736, 0.24032024, 0.47350714, -0.00336...",2


In [ ]:
df_val, df_test = train_test_split(df_val, test_size=0.4, random_state=35)

# LSTM: Creating the Final Model

In [ ]:
from keras import layers
text_input = Input(shape=(None,768,), dtype='float32', name='text')

l_mask = layers.Masking(mask_value=-99.)(text_input)
# Which we encoded in a single vector via a LSTM
encoded_text = layers.LSTM(100,)(l_mask)
out_dense = layers.Dense(30, activation='relu')(encoded_text)
# And we add a softmax classifier on top
out = layers.Dense(len(label_list), activation='softmax')(out_dense)
# At model instantiation, we specify the input and the output:
model = Model(text_input, out)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            (None, None, 768)         0         
_________________________________________________________________
masking_2 (Masking)          (None, None, 768)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               347600    
_________________________________________________________________
dense_3 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                310       
Total params: 350,940
Trainable params: 350,940
Non-trainable params: 0
_________________________________________________________________


In [ ]:
df_train.shape, df_val.shape, df_test.shape

((13713, 2), (2057, 2), (1372, 2))

The generator functions:

In [ ]:
num_sequences = len(df_train['emb'].to_list())
batch_size = 3
batches_per_epoch =  4571
assert batch_size * batches_per_epoch == num_sequences
num_features= 768
def train_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch):
            longest_index = (b + 1) * batch_size - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size][-batch_size:], key=len))
            x_train = np.full((batch_size, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size,  1))
            for i in range(batch_size):
                li = b * batch_size + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
num_sequences_val = len(df_val['emb'].to_list())
batch_size_val = 11
batches_per_epoch_val = 187
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
def val_generator(df):
    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size_val][-31:], key=len))
            # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
            x_train = np.full((batch_size_val, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                # print("li", li)
                # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

# LSTM Final Model: Training

In [ ]:
from keras.callbacks import ReduceLROnPlateau
call_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3, verbose=2,
                                mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

In [ ]:
model.fit_generator(train_generator(df_train), steps_per_epoch=batches_per_epoch, epochs=10,
                    validation_data=val_generator(df_val), validation_steps=batches_per_epoch_val, callbacks =[call_reduce] )

Epoch 1/10
4571/4571 [==============================] - 61s 13ms/step - loss: 0.3352 - acc: 0.9113 - val_loss: 0.3816 - val_acc: 0.8979
Epoch 2/10
4571/4571 [==============================] - 60s 13ms/step - loss: 0.3059 - acc: 0.9150 - val_loss: 0.3866 - val_acc: 0.8906
Epoch 3/10
4571/4571 [==============================] - 61s 13ms/step - loss: 0.2946 - acc: 0.9184 - val_loss: 0.3774 - val_acc: 0.8960
Epoch 4/10
4571/4571 [==============================] - 59s 13ms/step - loss: 0.2865 - acc: 0.9189 - val_loss: 0.3817 - val_acc: 0.8969

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009500000451225787.
Epoch 5/10
4571/4571 [==============================] - 60s 13ms/step - loss: 0.2805 - acc: 0.9202 - val_loss: 0.3752 - val_acc: 0.8979
Epoch 6/10
4571/4571 [==============================] - 60s 13ms/step - loss: 0.2748 - acc: 0.9210 - val_loss: 0.3730 - val_acc: 0.8960
Epoch 7/10
4571/4571 [==============================] - 59s 13ms/step - loss: 0.2720 - acc: 0.9215 - va

# LSTM Final Model: Evaluation



In [ ]:
num_sequences_val = len(df_test['emb'].to_list())
batch_size_val = 4
batches_per_epoch_val = 343
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
model.evaluate_generator(val_generator(df_test), steps= batches_per_epoch_val)

[0.41612950315069047, 0.8731778425655977]